# Analyze Samples
## Imports

In [ ]:
import os
import random

from tlib import tlearn, tgeo, tutils
from asos import settings, utils

%load_ext autoreload
%autoreload 2

## Setup Files and Plotter

In [ ]:
channel_indices = {'B2': 0, 'B3': 1, 'B4': 2, 'B5': 3, 'B6': 4, 'B7': 5, 'B8': 6, 'B8A': 7, 'B11': 8, 'B12': 9}

either files from dataset(s)...

In [ ]:
# files from dataset
datasets = ['test']

# get files from given dataset
csv = settings.load_file_infos()
file_infos = csv.df
file_infos = file_infos[file_infos['dataset'].isin(datasets)]
print(f'# files: {len(file_infos)}')
files = file_infos.index

# edit file names
files = list(files)
files = ['/'.join(file.split('/')[1:]) for file in files]  # remove True / False folder in the beginning
files = ['_'.join(file.split('_')[:-1]) + '.tif' for file in files]  # remove tile number in the end
print(len(files))
files = list(set(files))  # remove doubles
print(f'# files: {len(files)}')
files = [os.path.join(settings.data_folder_raw, file) for file in files]  # add file directory in front of files

nearby_files_possible = True  # only if file_infos is available
plot_predictions = False  # only if file_infos is available
batch_size = 1

# setup s2 plotter
s2_plotter = tgeo.s2.Plotter(folder_s2=settings.data_folder_raw, channel_indices=channel_indices)
s2_plotter.get_masks_dir = lambda file: os.path.join(os.path.join(settings.data_folder_raw, 'mask'), file.split('/')[-1])

# setup asos plotter
asos_plotter = utils.Plotter(folder=settings.data_folder_tiles_main)

... or specific file(s):

In [ ]:
# specific file

#folder = settings.data_folder_tiles_main
folder = os.path.join(settings.data_folder_raw, 's2_summer')

regex_filter = '.tif'  # e.g. '.tif'
smaller_than = 1000  # MB (up to 1000)


# get files from given folder and regex_filter
files = tutils.files.files_from_folder(folder, regex_filter=regex_filter, whole_path=True)
print(f'# files: {len(files)}')
# only files smaller than...
files = [file for file in files if os.path.getsize(file)/1000/1000 < smaller_than]
print(f'# files smaller than given size: {len(files)}')

nearby_files_possible = False  # only if file_infos is available
plot_predictions = False  # only if file_infos is available
batch_size = 1  # since sizes can be different

# setup plotter
s2_plotter = tgeo.s2.Plotter(folder_s2=folder, channel_indices=channel_indices)
asos_plotter = utils.Plotter(folder=folder)

In [ ]:
files = files[:2]

## Plot Sample

In [ ]:
# get random sample from given dataset
index = random.sample(range(len(files)), 1)[0]
file = files[index]
#print(f'label: {int(file_infos[file_infos.index == file]["label"])}')
#print(f'pred:  {int(file_infos[file_infos.index == file]["pred"])}')

# plotter
s2_plotter.plot(
    file,

    plot_s2=True,
    #plot_s2_channels=True,
    #plot_masks=True,
)

asos_plotter.plot(
    file,
    
    plot_unet_maps=True,
    #plot_all_unet_maps=True,
    plot_sensitivities=True,
    #plot_osm=True,
    
    #fig_height=4.6,
)

## All Files to RGB Tiff
Note: If you do not have a trained model, yet, you can ignore some of the plot_kwargs! To plot them, you need to predict the image.

In [ ]:
output_folder = os.path.join(settings.working_folder, 'rgbs')

In [ ]:
# plotter
s2_plotter.val_range_s2 = (0, 2**10.5)
s2_plotter.to_tif(
    *files,
    output_folder=output_folder,
    
    plot_rgb=True,
    plot_false_colors=True,
    #plot_channels=['B6'],
    plot_ndvi=True,
    plot_evi=True,
    plot_masks=['BB'],
)

asos_plotter.to_tif(
    *files,
    output_folder=output_folder,
    
    plot_unet_maps=True,
    #plot_all_unet_maps=True,
    plot_sensitivities=True,
    #plot_osms=True,

    batch_size=batch_size,
)

## Create KML File

In [ ]:
if plot_predictions:
    kml = csv.preds_to_kml(files=file_infos.index)
else:
    kml = None

# add white background
#kml = tgeo.kml.background_to_kml(output_dir=output_folder, kml=kml)

kml = s2_plotter.tif_to_kml(main_folder=output_folder, kml=kml)

## Sample in Activation Space

In [ ]:
asos = utils.load_asos()
files

In [ ]:
index = 0
unet_map = utils.predict(files[index])
vectors = tlearn.interpret.chspace.vectorize(array=unet_map, axis=1, random_frac=0.01)
asos.plot_chspace(vectors=vectors, colors='rgb', plotly=True)
print()